Latest update: 2022-12-06 20:35:03.967692

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Train Object Detection models</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>

# 1. Set up and requirements

### Install and import Python packages

In [ ]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
    !pip install -q --upgrade pip
    !pip install -q -r koster_yolov4/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("koster_yolov4/tutorials")
    !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py
    !cp ../src/track.py ../yolov5_tracker/track.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys
    import pkgutil

    if pkgutil.find_loader("torch") is not None:
        !pip install -q --upgrade pip
        !pip install -q torch==1.8.0 torchvision==0.9.0
    # Replace nearest neighbours script with custom version (due to relative path issue)
    #!cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py
    #!cp ../src/track.py ../yolov5_tracker/track.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")

In [ ]:
# Set the directory of the libraries
import sys, os

sys.path.append("..")

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Import required modules
from pathlib import Path
from ipyfilechooser import FileChooser
import kso_utils.tutorials_utils as t_utils
import kso_utils.server_utils as s_utils
import kso_utils.project_utils as p_utils
import kso_utils.t3_utils as t3
import kso_utils.t4_utils as t4
import kso_utils.t5_utils as t5
import kso_utils.t6_utils as t6
import kso_utils.t8_utils as t8
from kso_utils.yolo_utils import frame_aggregation
from kso_utils.zooniverse_utils import populate_agg_annotations
import wandb

# Model-specific imports
import yolov5.train as train
import yolov5.val as val
import yolov5.detect as detect

clear_output()
print("Packages loaded successfully")

Packages loaded successfully


# 2. Train the model

🔴 <span style="color:red">&nbsp;NOTE: To be able to train your own models, you will need access to the Koster WANDB group. You may request this access by contacting jurie.germishuys@combine.se. </span>

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Template project', 'Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'…

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

INFO:root:Template project loaded succesfully
Template project loaded succesfully


### Configure data paths

In [ ]:
# Specify path containing the images and labels folders.
output_folder = t_utils.choose_folder(
    project.photo_folder if not project.photo_folder == "None" else ".", "output"
)

FileChooser(path='.', filename='', title='HTML(value='Choose location of output')', show_hidden='False', use_d…

🔴 <span style="color:red">&nbsp;NOTE: To be able to train your own models, your data_path must contain a yml file for data and hyperparameters. See https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data#11-create-datasetyaml  </span>

In [10]:
# Fix important paths
data_path, hyps_path = t5.setup_paths(output_folder.selected)
project_path = str(Path(output_folder.selected, project.Project_name.lower()))

INFO:root:Success! Paths to data.yaml and hyps.yaml found.
Success! Paths to data.yaml and hyps.yaml found.


### Choose a suitable experiment name

In [11]:
exp_name = t5.choose_experiment_name()

Text(value='exp_name', description='Experiment name:', placeholder='Choose an experiment name', style=Descript…

### Choose model to use for training

In [12]:
# Specify path to download baseline model
download_folder = t_utils.choose_folder(
    project.photo_folder if not project.photo_folder == "None" else ".",
    "model download",
)

FileChooser(path='.', filename='', title='HTML(value='Choose location of model download')', show_hidden='False…

In [13]:
weights = t5.choose_baseline_model(download_folder.value)

Dropdown(description='Select model:', layout=Layout(width='50%'), options=(('yolov5m-classifier', <ArtifactCol…

Output()

### Train model with given configuration

In [14]:
batch_size, epochs, conf_thres = t5.choose_train_params()

In [15]:
# Feel free to play around with the img_size parameter, which should work well by default.
train.run(
    entity="koster",
    data=data_path,
    hyp=hyps_path,
    weights=weights.artifact_path,
    project=project_path,
    name=exp_name.value,
    img_size=[720, 540],
    batch_size=int(batch_size.value),
    epochs=epochs.value,
    workers=1,
    single_cls=False,
    cache_images=True,
)

train: weights=artifacts/baseline-yolov5:v0/yolov5m.pt, cfg=, data=/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/ml-template-data/Koster_Seafloor_Obs_14:28:20.yaml, hyp=/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/ml-template-data/hyp.yaml, epochs=1, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=1, project=/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/ml-template-data/template project, name=exp_name, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=koster, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, img_size=[720, 540], cache_images=True
github: ⚠️ YOLOv5 is out of date by 24 commits. Use `git pull` or 

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Focus                     [3, 48, 3]                    
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  6    629760  models.common.C3                        [192, 192, 6]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]             

TypeError: run() got an unexpected keyword argument 'batch_size'

# 3. Evaluate model performance

In [ ]:
# Choose model
eval_model = FileChooser(project_path)
display(eval_model)

In [ ]:
# Find trained model weights
tuned_weights = f"{Path(project_path, eval_model.selected, 'weights', 'best.pt')}"

In [ ]:
# Evaluate YOLO Model on Unseen Test data
val.run(
    data=data_path,
    weights=tuned_weights,
    conf_thres=conf_thres.value,
    imgsz=640,
    half=False,
)

# (Optional) : 4. Enhance annotations using trained model

Enhancement uses the trained model to increase the amount of annotations in the training data. This should only be done in cases where it is absolutely necessary as bad predictions lead to worse predictions when used to train the next iteration of the model. 


🔴 <span style="color:red">&nbsp;NOTE: We recommend using a relatively high confidence threshold when enhancing trained models as low confidence predictions could significantly impact the quality of your annotated data.  </span>

In [ ]:
detect.run(
    weights=tuned_weights,
    source=output_folder.selected + "/images",
    imgsz=[640, 640],
    conf_thres=0.02,
    save_txt=True,
)

### Choose run to use as enhanced annotations

In [ ]:
runs = FileChooser(".")
display(runs)

In [ ]:
!mv {output_folder}"/labels" {output_folder}"/labels_org"
!mv {runs.selected}"/labels" {output_folder}"/labels"

#### Once you have moved the new labels to the original label location, you can return to Step 2 and train your model again. 

In [ ]:
# END